In [4]:
from tkinter import *

# Main class
class Application:
    # Constructor
    def __init__(self, master):
        cvt_from = StringVar()
        cvt_to = StringVar()
        cvt1_to = StringVar()
        cvt2_to = StringVar()
        cvt3_to = StringVar()  
        cvt4_to = StringVar()
        cvt5_to = StringVar()
        cvt6_to = StringVar()
        def LZW_compress():
            uncompressed = cvt_from.get()
            LZW_before_compression = len(uncompressed) * 8
            LZW_after_compression = 0      
            dict_size = 256
            dictionary = dict((chr(i), i) for i in range(dict_size))
            w = ""
            result = []
            for c in uncompressed:
                wc = w + c
                if wc in dictionary:
                    w = wc
                else:
                    result.append(dictionary[w])
                    dictionary[wc] = dict_size
                    dict_size += 1
                    w = c

            if w:
                result.append(dictionary[w])  
            sum=0
            for i in result:
                i=str(i)
                n=len(i)
                sum+=n
            LZW_after_compression=sum
            LZW_save_percentage=(LZW_before_compression - LZW_after_compression)/LZW_before_compression
            cvt_to.set('%s' % result)
            cvt2_to.set('%s before compression' % LZW_before_compression)
            cvt3_to.set('%s after compression' % LZW_after_compression)
            cvt4_to.set('%s save percentage' % LZW_save_percentage)
            
            print(result)
            return result

        def lzw_decompress():
            compressed = LZW_compress()
            
            from io import StringIO
            dict_size = 256
            #dict_size = 128
            dictionary = {i: chr(i) for i in range(dict_size)}
            
            result = StringIO()
            

            #w = chr(int(compressed.pop(0)))
            w = chr(compressed.pop(0))

            result.write(w)
    
            for k in compressed:
                if k in dictionary:
                    entry = dictionary[k]

                elif k == dict_size:
                    entry = w + w[0]
                else:
                    raise ValueError('Bad compressed k: %s' % k)

                result.write(entry)

                dictionary[dict_size] = w + entry[0]
                dict_size += 1
                w = entry
            b=result.getvalue()
            cvt1_to.set('%s LZW' % b)
            
            
        def RLE_encode_message():
            message = cvt_from.get()
            RLE_before_compression = len(message) * 8
            RLE_after_compression = 0
            global encoded_string 
            encoded_string = ""
            i = 0
            while (i <= len(message)-1):
                count = 1
                ch = message[i]
                j = i
                while (j < len(message)-1): 
                    if (message[j] == message[j + 1]): 
                        count = count + 1
                        j = j + 1
                    else: 
                        break
                encoded_string = encoded_string + str(count) + ch
                i = j + 1
            
            RLE_after_compression = len(encoded_string) * 8
            RLE_save_percentage=(RLE_before_compression - RLE_after_compression)/RLE_before_compression
            cvt_to.set('%s RLE' % encoded_string)
            cvt2_to.set('%s before compression' % RLE_before_compression)
            cvt3_to.set('%s after compression' % RLE_after_compression)
            cvt4_to.set('%s save_percentage' % RLE_save_percentage)
            
        def RLE_decode_message():
            our_message = encoded_string
            decoded_message = ""
            i=0
            j=0
            while (i <= len(our_message)-1):
                run_count = int(our_message[i])
                run_word = our_message[i + 1]
                for j in range(run_count):
                    decoded_message = decoded_message+run_word
                    j = j + 1
                i = i + 2
            cvt1_to.set('%s RLE' % decoded_message)
            
            
        class Node:
            def __init__(self, prob, symbol, left=None, right=None):
                self.prob = prob

                self.symbol = symbol

                self.left = left

                self.right = right

                self.code = ''

        codes = dict()

        def Calculate_Codes(node, val=''):
            newVal = val + str(node.code)

            if(node.left):
                Calculate_Codes(node.left, newVal)
            if(node.right):
                Calculate_Codes(node.right, newVal)

            if(not node.left and not node.right):
                codes[node.symbol] = newVal

            return codes        

        def Calculate_Probability(data):
            symbols = dict()
            for element in data:
                if symbols.get(element) == None:
                    symbols[element] = 1
                else: 
                    symbols[element] += 1     
            return symbols

        def Output_Encoded(data, coding):
            encoding_output = []
            for c in data:
                encoding_output.append(coding[c])

            string = ''.join([str(item) for item in encoding_output])    
            return string       


        def Huffman_Encoding():
            data=cvt_from.get()
            symbol_with_probs = Calculate_Probability(data)
            symbols = symbol_with_probs.keys()
            probabilities = symbol_with_probs.values()

            nodes = []

            for symbol in symbols:
                nodes.append(Node(symbol_with_probs.get(symbol), symbol))

            while len(nodes) > 1:
                nodes = sorted(nodes, key=lambda x: x.prob)
                right = nodes[0]
                left = nodes[1]

                left.code = 0
                right.code = 1

                newNode = Node(left.prob+right.prob, left.symbol+right.symbol, left, right)

                nodes.remove(left)
                nodes.remove(right)
                nodes.append(newNode)
            
            global huffman_encoding  
            huffman_encoding = Calculate_Codes(nodes[0])
            global tree
            tree = nodes[0]
            global encoded_huffman
            encoded_output = Output_Encoded(data,huffman_encoding)
            encoded_huffman = encoded_output
            hf_before_compression = len(data) * 8 
            hf_after_compression = 0
            symbols = huffman_encoding.keys()
            for symbol in symbols:
              count = data.count(symbol)
              hf_after_compression += count * len(huffman_encoding[symbol])
            hf_save_percentage=(hf_before_compression - hf_after_compression)/hf_before_compression
            cvt_to.set('%s Huffman Coding' % encoded_output)
            cvt2_to.set('%s before compression' % hf_before_compression)
            cvt3_to.set('%s after compression' % hf_after_compression)
            cvt4_to.set('%s save percentage' % hf_save_percentage)
            

            return encoded_output, nodes[0]  

        
        def Huffman_Decoding():
            encoded_data = encoded_huffman
            huffman_tree = tree
            tree_head = huffman_tree
            decoded_output = []
            for x in encoded_data:
                if x == '1':
                    huffman_tree = huffman_tree.right   
                elif x == '0':
                    huffman_tree = huffman_tree.left
                try:
                    if huffman_tree.left.symbol == None and huffman_tree.right.symbol == None:
                        pass
                except AttributeError:
                    decoded_output.append(huffman_tree.symbol)
                    huffman_tree = tree_head
            string = ''.join([str(item) for item in decoded_output])
            cvt1_to.set('%s Huffman Decoding' % string)
            return string 
   
        frame = Frame(master)
        frame.pack()
		
	# Label to display instructions
        lbl_one = Label(master, text='Enter the Codon Sequence \n(A,U,C,G)', font='freesansbold')
        lbl_one.pack(fill=X)

        # Take the input from the user
        freq_input = Entry(master, textvariable=cvt_from, relief='sunken', justify='center', width=30, font=14)
        freq_input.pack()

        # Label to display instructions
        lbl_two = Label(master, text='Click the Button \nTo Compress any \nCodon Sequence', font='freesansbold')
        lbl_two.pack(fill=X)
		
	# Button to do encoding
        
        btn_1 = Button(master, padx=8,pady=8,bd=8,fg='black', font=('arial',12,'bold'), text='RLE Compression', bg='RosyBrown1', command=RLE_encode_message)
        btn_1.pack(fill=X)
        
        btn_2 = Button(master, padx=8,pady=8,bd=8,fg='black', font=('arial',12,'bold'), text='LZW Compression', bg='RosyBrown2', command=LZW_compress)
        btn_2.pack(fill=X)
        
        btn_3 = Button(master, padx=8,pady=8,bd=8,fg='black', font=('arial',12,'bold'), text='Huffman_Encoding', bg='RosyBrown3', command=Huffman_Encoding)
        btn_3.pack(fill=X)
	# Label to display the results of the calculation	
        lbl_result = Label(master, textvariable=cvt_to, relief='flat', bg='pale turquoise', font='freesansbold', fg='black')
        lbl_result.pack(fill=BOTH, expand=1)
        lbl_result1 = Label(master, textvariable=cvt2_to, relief='flat', bg='pale turquoise', font='freesansbold', fg='black')
        lbl_result1.pack(fill=BOTH, expand=1)
        lbl_result2 = Label(master, textvariable=cvt3_to, relief='flat', bg='pale turquoise', font='freesansbold', fg='black')
        lbl_result2.pack(fill=BOTH, expand=1)
        lbl_result3 = Label(master, textvariable=cvt4_to, relief='flat', bg='pale turquoise', font='freesansbold', fg='black')
        lbl_result3.pack(fill=BOTH, expand=1)

	# Label to display instructions
        lbl_one = Label(master, text='Enter the encoded codon sequence \n(RLE, LZW or Huffman)', font='freesansbold')
        lbl_one.pack(fill=X)

        # Take input from the encoded output
        freq_input = Entry(master, textvariable=cvt_to, relief='sunken', justify='center', width=30, font=14)
        freq_input.pack()

        # Label to display instructions
        lbl_two = Label(master, text='Click the Button \nTo Compress any \nCodon Sequence', font='freesansbold')
        lbl_two.pack(fill=X)
		
	# Button to select decoding algorithm
        
        btn_1 = Button(master, padx=8,pady=8,bd=8,fg='black', font=('arial',12,'bold'), text='RLE DeCompression', bg='RosyBrown1', command=RLE_decode_message)
        btn_1.pack(fill=X)
        
        btn_2 = Button(master, padx=8,pady=8,bd=8,fg='black', font=('arial',12,'bold'), text='LZW DeCompression', bg='RosyBrown2', command=lzw_decompress)
        btn_2.pack(fill=X)
        btn_3 = Button(master, padx=8,pady=8,bd=8,fg='black', font=('arial',12,'bold'), text='Huffman Decoding', bg='RosyBrown3', command=Huffman_Decoding)
        btn_3.pack(fill=X)
	# Label to display the results of the calculation	
        lbl_result = Label(master, textvariable=cvt1_to, relief='flat', bg='pale turquoise', font='freesansbold', fg='black')
        lbl_result.pack(fill=BOTH, expand=1)
		

root = Tk()
app = Application(root)
root.title('Lossless data compression')
root.minsize(350,350)

root.mainloop()


[116, 256]
[116, 256]
